In [84]:
import pandas as pd
import numpy as np

from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer

from itertools import combinations
from functools import cmp_to_key
import warnings

# Apriori

In [2]:
@cmp_to_key
def _itemset_str_comparator(a:str, b: str):
        if len(a) < len(b):
            return -1
        if len(a) > len(b):
            return 1

        if a < b:
            return -1
        if a > b:
            return 1
        return 0

In [3]:
class AprioriSolver:
    def __init__(self, data: list, minsup: int):
        self.data = [set(str(x)) for x in data]
        self.minsup = minsup
        self._unique_attributes = set.union(*self.data)

    def _support(self, itemset: set):
        res = 0
        for transaction in self.data:
            if itemset.issubset(transaction):
                res += 1
        return res

    def _freq_itemsets_bruteforce(self, size=1):
        res = []
        for combo in combinations(self._unique_attributes, size):
            itemset = set(combo)
            if self._support(itemset) >= self.minsup:
                res.append(itemset)

        return res

    def apriori_gen(self, L_k1, k):
        C_k = []
        for p in L_k1:
            for q in L_k1:
                if k > 2 and list(p)[:k-2] != list(q)[:k-2]:
                    continue
                if k > 1 and list(p)[k-2] >= list(q)[k-2]:
                    continue
                C_k.append(set.union(p, q))

        bad_cs = []
        for c in C_k:
            for item in c:
                subset = c.difference({item})
                if subset not in L_k1:
                    bad_cs.append(c)

        for bad_c in bad_cs:
            C_k.remove(bad_c)

        return C_k

    def apriori(self, verbose=False):
        L = [None, self._freq_itemsets_bruteforce()]
        C = [None, [{x} for x in self._unique_attributes]]
        k = 2
        while len(L[k-1]) > 0:
            C_k = self.apriori_gen(L[k-1], k)
            L_k = []

            for c in C_k:
                if self._support(c) >= self.minsup:
                    L_k.append(c)

            L.append(L_k)
            C.append(C_k)

            k += 1

        self.Ls = L
        self.Cs = C

    def print_report(self):
        print("Main Apriori Process:")
        for i in range(1, len(self.Ls)):
            C_dict, L_dict = dict(), dict()
            for c in self.Cs[i]:
                C_dict["".join(sorted(c))] = self._support(c)
            for l in self.Ls[i]:
                L_dict["".join(sorted(l))] = self._support(l)

            keys = list(sorted(C_dict.keys()))
            C_dict = {key: C_dict[key] for key in keys}

            keys = list(sorted(L_dict.keys()))
            L_dict = {key: L_dict[key] for key in keys}

            print(f"C{i}: {C_dict}")
            print(f"L{i}: {L_dict}")
            print("=============")

    def _format_itemset_list_as_dict(self, l: list):
        res_dict = dict()
        for c in l:
            res_dict["".join(sorted(c))] = self._support(c)

        keys = list(sorted(res_dict.keys(), key=_itemset_str_comparator))
        res_dict = {key: res_dict[key] for key in keys}
        return res_dict

    def _is_superset_in_list(self, s: set, l: list):
        for elem in l:
            if elem.issuperset(s):
                return True
        return False

    def get_maximal_frequent(self):
        result = []
        for i in range(1, len(self.Ls)-1):
            for itemset in self.Ls[i]:
                if not self._is_superset_in_list(itemset, self.Ls[i+1]):
                    result.append(itemset)
        result.extend(self.Ls[-1])
        result = self._format_itemset_list_as_dict(result)
        return result
        

    def get_closed_frequent(self):
        result = []
        for i in range(1, len(self.Ls)-1):
            for itemset in self.Ls[i]:
                good = True
                for upper in self.Ls[i+1]:
                    if upper.issuperset(itemset) and self._support(itemset) == self._support(upper):
                        good = False
                        break
                if good:
                    result.append(itemset)
        result.extend(self.Ls[-1])
        result = self._format_itemset_list_as_dict(result)
        return result

### Example 1

In [4]:
data = ['ABDE', 'BCE', 'ABDE', 'ABCE', 'ABCDE', 'BCD']
apriorisolver = AprioriSolver(data, minsup=2)
apriorisolver.apriori()
apriorisolver.print_report()

Main Apriori Process:
C1: {'A': 4, 'B': 6, 'C': 4, 'D': 4, 'E': 5}
L1: {'A': 4, 'B': 6, 'C': 4, 'D': 4, 'E': 5}
C2: {'AB': 4, 'AC': 2, 'AD': 3, 'AE': 4, 'BC': 4, 'BD': 4, 'BE': 5, 'CD': 2, 'CE': 3, 'DE': 3}
L2: {'AB': 4, 'AC': 2, 'AD': 3, 'AE': 4, 'BC': 4, 'BD': 4, 'BE': 5, 'CD': 2, 'CE': 3, 'DE': 3}
C3: {'ABC': 2, 'ABD': 3, 'ABE': 4, 'ACD': 1, 'ACE': 2, 'ADE': 3, 'BCD': 2, 'BCE': 3, 'BDE': 3, 'CDE': 1}
L3: {'ABC': 2, 'ABD': 3, 'ABE': 4, 'ACE': 2, 'ADE': 3, 'BCD': 2, 'BCE': 3, 'BDE': 3}
C4: {'ABCE': 2, 'ABDE': 3}
L4: {'ABCE': 2, 'ABDE': 3}
C5: {}
L5: {}


In [5]:
apriorisolver.get_closed_frequent()

{'B': 6,
 'BC': 4,
 'BD': 4,
 'BE': 5,
 'ABE': 4,
 'BCD': 2,
 'BCE': 3,
 'ABCE': 2,
 'ABDE': 3}

In [6]:
apriorisolver.get_maximal_frequent()

{'BCD': 2, 'ABCE': 2, 'ABDE': 3}

In [7]:
data = ["136A", '136A', '1479', '2479', '2579', '2579', '2589', '136B']
apriorisolver = AprioriSolver(data, minsup=4)
apriorisolver.apriori()
apriorisolver.print_report()

Main Apriori Process:
C1: {'1': 4, '2': 4, '3': 3, '4': 2, '5': 3, '6': 3, '7': 4, '8': 1, '9': 5, 'A': 2, 'B': 1}
L1: {'1': 4, '2': 4, '7': 4, '9': 5}
C2: {'12': 0, '17': 1, '19': 1, '27': 3, '29': 4, '79': 4}
L2: {'29': 4, '79': 4}
C3: {}
L3: {}


# AHC

In [88]:
def measure(a, b, measure_name):
    """ Compute the dist or similarity between two sets """
    # a b are numpy arrays
    if measure_name == 'jaccard':
        # only consider where both are 1
        intersection = np.logical_and(a, b)
        union = np.logical_or(a, b)
        return - intersection.sum() / union.sum()
    elif measure_name == 'euclidean':
        return np.linalg.norm(np.array(a) - np.array(b))
    elif measure_name == 'cosine':
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def compute_distance_matrix(data, measure_name):
    """ Compute and print the distance matrix for the data """
    data = MultiLabelBinarizer().fit_transform(data)
    labels = [('T'+str(i+1), ) for i in range(len(data))]
    square_distance_matrix = np.zeros((len(data), len(data)))
    for i in range(len(data)):
        for j in range(len(data)):
            if i == j:
                square_distance_matrix[i][j] = 0
            else:
                square_distance_matrix[i][j] = measure(data[i], data[j], measure_name)
    
    square_distance_matrix[np.eye(square_distance_matrix.shape[0], dtype=bool)] = np.inf
    square_distance_matrix *= -1
    df = pd.DataFrame(square_distance_matrix, columns=labels, index=labels)
    
    return df

In [85]:
class AHCSolver:
    def __init__(self, data: pd.DataFrame, dist_type="MIN"):
        self.data = data
        self.dist_type = dist_type
        self.history = [data]

    def _find_max(self, table):
        max_value = table.values.max()
        max_index = table.values.argmax()
        max_row, max_col = divmod(max_index, table.shape[1])
        max_index_label = table.index[max_row]
        max_column_label = table.columns[max_col]

        return max_value, max_index_label, max_column_label

    def _merge_clusters(self, table: pd.DataFrame, c1, c2):
        new_c = c1 + c2
        
        other_cs = table.index[(table.index != c1) & (table.index != c2)].to_list()
        new_labels = other_cs + [new_c]
        new_table = pd.DataFrame(table, columns=new_labels, index=new_labels)

        for column in new_table.columns.to_list():
            if column == new_c:
                new_table.loc[[new_c], [column]] = -np.inf
                continue
            if self.dist_type == "MIN":
                new_val = table.loc[[column], [c1, c2]].values.max()
            elif self.dist_type == "MAX":
                new_val = table.loc[[column], [c1, c2]].values.min()

            new_table.loc[[new_c], [column]] = new_val
        
        new_table.loc[:, [new_c]] = new_table.loc[[new_c], :].values.reshape((len(new_labels), 1))
        return new_table
        

    def ahc(self):
        with warnings.catch_warnings(record=True) as w:
            n_clusters = len(self.data)
            while n_clusters > 2:
                _, c1, c2 = self._find_max(self.history[-1])

                new_table = self._merge_clusters(self.history[-1], c1, c2)
                self.history.append(new_table)

                n_clusters -= 1

    def print_history(self):
        print("History of merging:")
        for table in self.history:
            print(table)
            print("=======================")

### Example 1

In [76]:
data_array = np.array([
    [1.0, 0.1, 0.41, 0.55, 0.35],
    [0, 1.0, 0.64, 0.47, 0.98],
    [0, 0, 1.0, 0.44, 0.85],
    [0, 0, 0, 1.0, 0.76],
    [0, 0, 0, 0, 1.0]
])

data_array += data_array.T
data_array[np.eye(data_array.shape[0], dtype=bool)] = -np.inf
data_array

labels = ["p1", "p2", "p3", "p4", "p5"]
labels = [(x,) for x in labels]
data = pd.DataFrame(data_array, index=labels, columns=labels)
data

,"(p1,)","(p2,)","(p3,)","(p4,)","(p5,)"
"(p1,)",-inf,0.10,0.41,0.55,0.35
"(p2,)",0.10,-inf,0.64,0.47,0.98
"(p3,)",0.41,0.64,-inf,0.44,0.85
"(p4,)",0.55,0.47,0.44,-inf,0.76
"(p5,)",0.35,0.98,0.85,0.76,-inf


In [78]:
ahcsolver = AHCSolver(data, dist_type="MAX")
ahcsolver.ahc()
ahcsolver.print_history()

### Example 2

In [90]:
raw_data = [
    ['A', 'B', 'C', 'D', 'E', 'F'],
    ['A', 'B', 'C', 'D', 'E'],
    ['B', 'C', 'D', 'E'],
    ['F', 'G'],
    ['A', 'F', 'G'],
    ['H', 'I']
]

matrix_data = compute_distance_matrix(data, 'jaccard')
solver = AHCSolver(matrix_data, dist_type="MIN")
solver.ahc()
solver.print_history()

History of merging:
          (T1,)     (T2,)     (T3,)     (T4,)     (T5,)     (T6,)
(T1,)      -inf  0.833333  0.666667  0.142857  0.285714  0.125000
(T2,)  0.833333      -inf  0.800000 -0.000000  0.142857 -0.000000
(T3,)  0.666667  0.800000      -inf -0.000000 -0.000000 -0.000000
(T4,)  0.142857 -0.000000 -0.000000      -inf  0.666667  0.666667
(T5,)  0.285714  0.142857 -0.000000  0.666667      -inf  0.500000
(T6,)  0.125000 -0.000000 -0.000000  0.666667  0.500000      -inf
          (T3,)     (T4,)     (T5,)     (T6,)  (T1, T2)
(T3,)      -inf -0.000000 -0.000000 -0.000000  0.800000
(T4,)      -0.0      -inf  0.666667  0.666667  0.142857
(T5,)      -0.0  0.666667      -inf  0.500000  0.285714
(T6,)      -0.0  0.666667  0.500000      -inf  0.125000
(T1, T2)    0.8  0.142857  0.285714  0.125000      -inf
                 (T4,)     (T5,)     (T6,)  (T3, T1, T2)
(T4,)             -inf  0.666667  0.666667      0.142857
(T5,)         0.666667      -inf  0.500000      0.285714
(T6,)      